In [1]:
import os

In [4]:
import os
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [17]:
(X_train_full, y_train_full),(X_test, y_test)=tf.keras.datasets.mnist.load_data()
X_train_full=X_train_full/255.0
X_test=X_test/255.0
X_valid,X_train=X_train_full[:5000],X_train_full[5000:]
y_valid,y_train=y_train_full[:5000],y_train_full[5000:]


In [18]:
tf.random.set_seed(42)
#tf.random.seed(42)
LAYERS=[
    tf.keras.layers.Flatten(input_shape=[28,28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")
    
]

In [19]:
model=tf.keras.models.Sequential(LAYERS)

In [20]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(lr=1e-3), metrics=["accuracy"])

D:\DLCVANLP\Deep-Learning\env\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [16]:
#del model

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 300)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-tr

In [22]:
history=model.fit(X_train,y_train, epochs=10, validation_data=(X_valid,y_valid), verbose=2)

Epoch 1/10
1719/1719 - 4s - loss: 1.5275 - accuracy: 0.5970 - val_loss: 0.9444 - val_accuracy: 0.7980 - 4s/epoch - 2ms/step
Epoch 2/10
1719/1719 - 4s - loss: 0.7465 - accuracy: 0.8287 - val_loss: 0.5868 - val_accuracy: 0.8596 - 4s/epoch - 2ms/step
Epoch 3/10
1719/1719 - 4s - loss: 0.5412 - accuracy: 0.8624 - val_loss: 0.4685 - val_accuracy: 0.8834 - 4s/epoch - 2ms/step
Epoch 4/10
1719/1719 - 4s - loss: 0.4591 - accuracy: 0.8771 - val_loss: 0.4104 - val_accuracy: 0.8940 - 4s/epoch - 2ms/step
Epoch 5/10
1719/1719 - 4s - loss: 0.4142 - accuracy: 0.8869 - val_loss: 0.3758 - val_accuracy: 0.9006 - 4s/epoch - 2ms/step
Epoch 6/10
1719/1719 - 4s - loss: 0.3851 - accuracy: 0.8938 - val_loss: 0.3525 - val_accuracy: 0.9052 - 4s/epoch - 2ms/step
Epoch 7/10
1719/1719 - 5s - loss: 0.3644 - accuracy: 0.8980 - val_loss: 0.3348 - val_accuracy: 0.9102 - 5s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 5s - loss: 0.3485 - accuracy: 0.9022 - val_loss: 0.3209 - val_accuracy: 0.9138 - 5s/epoch - 3ms/step
Epoch 9/

In [23]:
model.save("pretrained_mnist_model.h5")

# Transfer Learning
## New  Problem statement-----
Classify handwritten digits odd and even

In [24]:
#Loading pretrained model
pretrained_mnist_model=tf.keras.models.load_model("pretrained_mnist_model.h5")

In [25]:
pretrained_mnist_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 300)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-tr

In [26]:
for layer in pretrained_mnist_model.layers:
    print(f"{layer.name}: {layer.trainable}")
    

flatten_1: True
dense_3: True
leaky_re_lu_2: True
dense_4: True
leaky_re_lu_3: True
dense_5: True


In [27]:
for layer in pretrained_mnist_model.layers[:-1]:# leaves the last layer unfreezed =>tranable
    layer.trainable=False # freezing the layers from getting trained
    print(f"{layer.name}: {layer.trainable}")

flatten_1: False
dense_3: False
leaky_re_lu_2: False
dense_4: False
leaky_re_lu_3: False


In [28]:
for layer in pretrained_mnist_model.layers:
    print(f"{layer.name}: {layer.trainable}")

flatten_1: False
dense_3: False
leaky_re_lu_2: False
dense_4: False
leaky_re_lu_3: False
dense_5: True


In [31]:
try:
    del new_model
except:
    pass

In [32]:
lower_pretrained_layers=pretrained_mnist_model.layers[:-1]
# LAYERS=[
#     lower_pretrained_layers,
#     tf.keras.layers.Dense(2, activation="softmax")
    
# ]
new_model=tf.keras.models.Sequential(lower_pretrained_layers)
new_model.add(tf.keras.layers.Dense(2, activation="softmax"))

In [33]:
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 300)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_7 (Dense)             (None, 2)                 202       
                                                                 
Total params: 265,802
Trainable params: 202
Non-traina

In [34]:
100 * 2 + 2

202

In [35]:
265600+202

265802

In [36]:
# Non trainable parameter
235500+30100

265600

In [37]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [38]:
np.where(y_train%2==0,1,0)

array([0, 0, 1, ..., 0, 1, 1])

In [40]:
y_train[0], y_train[-1]

(7, 8)

In [41]:
def update_even_odd_labels(labels):
    for idx, label in enumerate(labels):
        labels[idx]=np.where(label%2==0,1,0)
    return labels

In [42]:
y_train_bin, y_test_bin, y_valid_bin= update_even_odd_labels([y_train,y_test,y_valid])

In [43]:
np.unique(y_train_bin)

array([0, 1])

In [44]:
new_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(lr=1e-3), metrics=["accuracy"])

D:\DLCVANLP\Deep-Learning\env\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [45]:
history=new_model.fit(X_train,y_train_bin, epochs=10, validation_data=(X_valid,y_valid_bin), verbose=2)

Epoch 1/10
1719/1719 - 6s - loss: 0.3898 - accuracy: 0.8288 - val_loss: 0.3247 - val_accuracy: 0.8676 - 6s/epoch - 3ms/step
Epoch 2/10
1719/1719 - 5s - loss: 0.3300 - accuracy: 0.8602 - val_loss: 0.3049 - val_accuracy: 0.8754 - 5s/epoch - 3ms/step
Epoch 3/10
1719/1719 - 5s - loss: 0.3163 - accuracy: 0.8660 - val_loss: 0.2948 - val_accuracy: 0.8796 - 5s/epoch - 3ms/step
Epoch 4/10
1719/1719 - 4s - loss: 0.3083 - accuracy: 0.8701 - val_loss: 0.2884 - val_accuracy: 0.8832 - 4s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 4s - loss: 0.3023 - accuracy: 0.8725 - val_loss: 0.2834 - val_accuracy: 0.8846 - 4s/epoch - 3ms/step
Epoch 6/10
1719/1719 - 4s - loss: 0.2978 - accuracy: 0.8752 - val_loss: 0.2792 - val_accuracy: 0.8872 - 4s/epoch - 2ms/step
Epoch 7/10
1719/1719 - 4s - loss: 0.2939 - accuracy: 0.8772 - val_loss: 0.2758 - val_accuracy: 0.8872 - 4s/epoch - 2ms/step
Epoch 8/10
1719/1719 - 4s - loss: 0.2907 - accuracy: 0.8788 - val_loss: 0.2728 - val_accuracy: 0.8890 - 4s/epoch - 2ms/step
Epoch 9/

In [46]:
new_model.evaluate(X_test,y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.2768 - accuracy: 0.8861


[0.27683505415916443, 0.8860999941825867]

In [47]:
X_new=X_test[:3]

In [48]:
np.argmax(new_model.predict(X_new), axis=-1)

array([0, 1, 0], dtype=int64)